# Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Reading the file
df = pd.read_csv('space_decay.csv')

 # Data Cleaning


Handling the Missing Values

In [ ]:
# Accessing heads
df.head(1)


In [ ]:
# checking for null values
df.isnull().sum()

In [ ]:
# if null values are found in decay_date change it
# The object is active if the decay_date is null
df['IS_ACTIVE'] = df['DECAY_DATE'].isna()
print(df['IS_ACTIVE'])

Invalid Value Detection

In [ ]:
# Checking for eccentricity 
#(It is checking for eccentricity values and updated the original column)
df = df[(df['ECCENTRICITY'] >= 0) & (df['ECCENTRICITY'] < 1)]
print(df['ECCENTRICITY'])

# Checking for inclination
df = df[(df['INCLINATION'] >= 0)& (df['INCLINATION']<=180)]
print(df['INCLINATION'])

# Checking for mean motion
df=df[(df['MEAN_MOTION']>=0)]
print(df['MEAN_MOTION'])

# Checking for semi-major axis
df=df[(df['SEMIMAJOR_AXIS']> 6378)]
print(df['SEMIMAJOR_AXIS'])

Standardization of Units and Formats

In [ ]:
# Converting all the dates in the standard format
 # Convert to datetime format
date_columns = ['EPOCH', 'CREATION_DATE', 'DECAY_DATE', 'LAUNCH_DATE']
for i in date_columns:
    df[i] = pd.to_datetime(df[i], errors='coerce')

## ORBITAL DYNAMICS AND DERIVED METRICS ##

# Altitude at Perigee / Apogee  
df['Perigee_Altitude'] = df['SEMIMAJOR_AXIS']*(1 - df['ECCENTRICITY']) - 6378.137
df['Apogee_Altitude'] = df['SEMIMAJOR_AXIS']*(1 + df['ECCENTRICITY']) - 6378.137
print(df['Perigee_Altitude'])
print(df['Apogee_Altitude'])

# Orbit Regime classification (LEO, MEO, GEO, HEO)
def classify_orbit_regime(Perigee_Altitude):
    if  Perigee_Altitude >= 160 and Perigee_Altitude<= 2000:
        return 'LEO'
    elif Perigee_Altitude> 2000 and Perigee_Altitude < 35786:
        return 'MEO'
    elif np.isclose(Perigee_Altitude, 35786, atol=100):
        return 'GEO'
    elif Perigee_Altitude > 35786:
        return 'HEO'
    else:
        return 'Unknown'

df['ORBIT_REGIME'] = df['Perigee_Altitude'].apply(classify_orbit_regime)
print(df['ORBIT_REGIME'])

# Orbit type based on inclination (Polar, Non-Polar)
def classify_orbit_inclination(INCLINATION):
  if INCLINATION < 90 :
    return 'Non Polar Orbit'
  else :
    return 'Polar orbit'
df['ORBIT_ORIENTATION'] = df['INCLINATION'].apply(classify_orbit_inclination)
print(df['ORBIT_ORIENTATION'])

# Exploratory Data Analysis (EDA)

 Object distribution by type (payload, rocket body or debris)

In [ ]:
# to check all the unique objects with OBJECT_TYPE
print(df['OBJECT_TYPE'].unique())
def classify_object_type(OBJECT_TYPE):
    if  OBJECT_TYPE == 'PAYLOAD':
        return 'PAYLOAD'
    elif OBJECT_TYPE == 'ROCKET BODY':
        return 'ROCKET BODY'
    elif OBJECT_TYPE == 'DEBRIS':
       return 'DEBRIS'
    else:
        return 'Unknown'

df['OBJECT_TYPE_CLASSIFICATION'] = df['OBJECT_TYPE'].apply(classify_object_type)
print(df['OBJECT_TYPE_CLASSIFICATION'])

 Plotting 

In [ ]:
# Plotting the different object types to generate a pie-chart
# Count the number of satellites, payload and debris
type_counts = df['OBJECT_TYPE'].value_counts()

# Make pie chart
plt.figure(figsize=(6,6))
plt.pie(type_counts, labels=type_counts.index, autopct='%1.1f%%', startangle=140, colors=['skyblue', 'lightcoral','red','yellow'])
plt.title('Object Distribution by Type (Satellite/Debris/payload/TBA)')
plt.show()


In [ ]:
# Bar plot for country wise ownership of objects
# Count objects per country
country_counts = df['COUNTRY_CODE'].value_counts().head(10)  # Top 10 countries
print(country_counts)
# Bar plot
plt.figure(figsize=(10,6))
sns.barplot(x=country_counts.index, y=country_counts.values, palette='viridis')
plt.title('Top 10 Countries by Number of Space Objects')
plt.xlabel('Country Code')
plt.ylabel('Number of Objects')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
df.head(1)

In [ ]:
#  to group objects by size and object_type
object_distribution_based_on_size = df.groupby(['RCS_SIZE','OBJECT_TYPE']).size().reset_index(name='COUNT')
print(object_distribution_based_on_size)

count_object = df['RCS_SIZE'].value_counts()
print(count_object)


In [ ]:

count_object = df['OBJECT_TYPE'].value_counts()
print(count_object)

# Bar plot
plt.figure(figsize=(10,6))
sns.barplot(x=count_object.index, y=count_object.values, palette='viridis')
plt.title(' Space Objects')
plt.xlabel('Object Type')
plt.ylabel('Number of Objects')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
print(df.columns)